<a href="https://colab.research.google.com/github/taaha3244/GANs/blob/main/GANs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

In [2]:
class Discriminator(nn.Module):
  def __init__(self,img_dim):
    super().__init__()
    self.disc=nn.Sequential(
        nn.Linear(img_dim,128),
        nn.LeakyReLU(0,1),
        nn.Linear(128,1),
        nn.Sigmoid()
    )
  def forward(self,x):
    return self.disc(x)

In [3]:
class Generator(nn.Module):
  def __init__(self,z_dim,img_dim):
    super().__init__()
    self.gen=nn.Sequential(
        nn.Linear(z_dim,256),
        nn.LeakyReLU(),
        nn.Linear(256,img_dim),
        nn.Tanh(),
    )
  def forward(self,x):
    return self.gen(x)

In [4]:
device='cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
lr=3e-4
z_dim=64
image_dim=28*28*1
batch_size=32
num_epochs=50

In [6]:
disc=Discriminator(image_dim).to(device)

In [7]:
gen=Generator(z_dim,image_dim).to(device)

In [8]:
fixed_noise=torch.randn((batch_size,z_dim)).to(device)

In [9]:
transforms=transforms.Compose(
    [transforms.ToTensor(),transforms.Normalize((0.1307),(0.3081))]
)

In [10]:
dataset=datasets.MNIST(root='/cntent',transform=transforms,download=True)

In [11]:
loader=DataLoader(dataset,batch_size=batch_size,shuffle=True)

In [12]:
opt_disc=optim.Adam(disc.parameters(),lr=lr)

In [13]:
opt_gen=optim.Adam(gen.parameters(),lr=lr)

In [14]:
criterion=nn.BCELoss()

In [15]:
writer_fake=SummaryWriter(f"runs/GAB_MNIST/fake")
writer_real=SummaryWriter(f"runs/GAB_MNIST/real")
step=0

In [18]:
for epoch in range(num_epochs):
  for batch_idx,(real,_) in enumerate(loader):
    real=real.view(-1,784).to(device)
    batch_size=real.shape[0]

    noise=torch.randn(batch_size,z_dim).to(device)
    fake=gen(noise)
    disc_real=disc(real).view(-1)
    lossD_real=criterion(disc_real,torch.ones_like(disc_real))
    disc_fake=disc(fake.detach()).view(-1)
    lossD_fake=criterion(disc_fake,torch.zeros_like(disc_fake))
    lossD=(lossD_real + lossD_fake)/2
    disc.zero_grad()
    lossD.backward()
    opt_disc.step()


    output=disc(fake).view(-1)
    lossG=criterion(output,torch.ones_like(output))
    gen.zero_grad()
    lossG.backward()
    opt_gen.step()


    if batch_idx==0:
      print(
          f"Epoch[{epoch}/{num_epochs}"
          f" LossD :{lossD:4f},Loss G: {lossG:.4f}]"
      )

      with torch.no_grad():
        fake=gen(fixed_noise).reshape(-1,1,28,28)
        data=real.reshape(-1,1,28,28)
        img_grid_fake=torchvision.utils.make_grid(fake,normalize=True)
        img_grid_real=torchvision.utils.make_grid(data,normalize=True)

        writer_fake.add_image(
            "MNIST Fake Images",img_grid_fake,global_step=step
        )

        writer_real.add_image(
            "MNIST real Images",img_grid_real,global_step=step
        )

        step+=1


tensorboard --logdir=real

Epoch[0/50 LossD :0.577064,Loss G: 0.6308]
Epoch[1/50 LossD :0.065219,Loss G: 3.0128]
Epoch[2/50 LossD :0.046104,Loss G: 3.5315]
Epoch[3/50 LossD :0.019892,Loss G: 4.5150]
Epoch[4/50 LossD :0.047554,Loss G: 4.8180]
Epoch[5/50 LossD :0.010108,Loss G: 6.1748]
Epoch[6/50 LossD :0.024788,Loss G: 4.4305]
Epoch[7/50 LossD :0.012588,Loss G: 5.1981]
Epoch[8/50 LossD :0.006823,Loss G: 6.1561]
Epoch[9/50 LossD :0.012583,Loss G: 5.7642]
Epoch[10/50 LossD :0.024802,Loss G: 5.7651]
Epoch[11/50 LossD :0.005035,Loss G: 6.2770]
Epoch[12/50 LossD :0.022170,Loss G: 5.9358]
Epoch[13/50 LossD :0.022987,Loss G: 6.3479]
Epoch[14/50 LossD :0.013539,Loss G: 5.8328]
Epoch[15/50 LossD :0.008152,Loss G: 5.7340]
Epoch[16/50 LossD :0.014114,Loss G: 6.5930]


KeyboardInterrupt: 